In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from collections import Counter
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, AutoConfig, get_linear_schedule_with_warmup
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import random
import warnings
from torch.optim import AdamW
warnings.filterwarnings('ignore')



# ==================== КОНФИГУРАЦИЯ ====================
class Config:
    model_name = "sberbank-ai/ruBert-base"
    batch_size = 16
    accumulation_steps = 2
    learning_rate = 1.5e-5  # Увеличил learning rate
    epochs = 5  # Увеличил количество эпох
    max_length = 300  # Увеличил длину текста
    patience = 2  # Ранняя остановка
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_warmup_ratio = 0.11  # Прогрев для шедулера

config = Config()

# ==================== КАСТОМНАЯ МОДЕЛЬ ====================
class MultiLabelTransformerWithHeads(nn.Module):
    """Кастомная модель с несколькими головками классификации"""
    def __init__(self, model_name, num_labels, hidden_dropout_prob=0.3):
        super().__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.backbone = AutoModel.from_pretrained(model_name)

        # Разные головки для разных групп категорий
        self.category_head = nn.Linear(self.config.hidden_size, num_labels)
        self.genre_head = nn.Linear(self.config.hidden_size, num_labels)

        self.dropout = nn.Dropout(hidden_dropout_prob)
        self.layer_norm = nn.LayerNorm(self.config.hidden_size)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]  # [CLS] token

        # Применяем разные головки и усредняем
        normalized_output = self.layer_norm(pooled_output)
        dropout_output = self.dropout(normalized_output)

        logits1 = self.category_head(dropout_output)
        logits2 = self.genre_head(dropout_output)

        # Усреднение предсказаний
        logits = (logits1 + logits2) / 2

        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels)

        return type('Output', (), {'loss': loss, 'logits': logits})

# ==================== УЛУЧШЕННАЯ ПОДГОТОВКА ДАННЫХ ====================
def load_and_prepare_data():
    """Загрузка и улучшенная подготовка данных"""
    train_data = pd.read_csv('/content/train.tsv', sep='\t')
    test_data = pd.read_csv('/content/test.tsv', sep='\t')

    print("Доступные колонки в train:", train_data.columns.tolist())

    # УЛУЧШЕНИЕ: Более умное создание текста с приоритетами
    def create_enhanced_text(row):
        parts = []

        # 1. Название приложения (самое важное)
        if 'app_name' in row:
            app_name = str(row['app_name']).strip()
            if app_name and app_name != 'nan':
                parts.append(f"Название: {app_name}")

        # 2. Короткое описание
        for col in ['shortDescription', 'short_description']:
            if col in row and pd.notna(row[col]):
                desc = str(row[col]).strip()
                if desc and desc != 'nan':
                    parts.append(f"Описание: {desc}")
                    break

        # 3. Полное описание
        for col in ['full_description', 'description', 'long_description']:
            if col in row and pd.notna(row[col]):
                full_desc = str(row[col]).strip()
                if full_desc and full_desc != 'nan':
                    # Берем только первые 200 символов полного описания
                    parts.append(f"Подробно: {full_desc[:200]}")
                    break

        # Если ничего нет, используем только название
        if not parts and 'app_name' in row:
            return str(row['app_name'])

        return " ".join(parts)

    train_data['text'] = train_data.apply(create_enhanced_text, axis=1)
    test_data['text'] = test_data.apply(create_enhanced_text, axis=1)

    # Анализ длины текстов
    train_data['text_length'] = train_data['text'].str.len()
    print(f"Средняя длина текста: {train_data['text_length'].mean():.0f} символов")
    print(f"Максимальная длина: {train_data['text_length'].max()} символов")

    # MultiLabelBinarizer для меток
    mlb = MultiLabelBinarizer()
    train_labels = mlb.fit_transform(train_data['labels_str'].str.split('|'))

    # Анализ распределения меток
    label_counts = train_labels.sum(axis=0)
    print(f"\n📊 Анализ меток:")
    print(f"Всего классов: {len(mlb.classes_)}")
    print(f"Общее количество меток: {train_labels.sum()}")
    print(f"Меток на приложение: {train_labels.sum(axis=1).mean():.2f}")

    # Выводим топ-10 самых частых меток
    top_labels = pd.DataFrame({
        'category': mlb.classes_,
        'count': label_counts
    }).sort_values('count', ascending=False).head(10)
    print("\nТоп-10 самых частых категорий:")
    print(top_labels)

    return train_data, test_data, mlb, train_labels

# ==================== УЛУЧШЕННЫЙ ДАТАСЕТ С АУГМЕНТАЦИЕЙ ====================
class AdvancedTextAugmenter:
    """Улучшенная аугментация текстов"""
    def __init__(self):
        self.synonyms = {
    'игра': ['гейм', 'игровой', 'игровое', 'развлечение', 'видеоигра', 'аркада', 'гейминг', 'игрушка', 'баталия', 'партия', 'состязание', 'забава', 'аттракцион', 'симулятор', 'квест', 'головоломка', 'стратегия', 'экшен', 'приключение'],
    'приложение': ['программа', 'софт', 'утилита', 'апп', 'приложенье', 'программка', 'софтина', 'программное обеспечение', 'ПО', 'аппликация', 'инструмент', 'программный продукт', 'софтвер', 'клиент', 'прога'],
    'бесплатный': ['фри', 'даром', 'без оплаты', 'бесплатно', 'free', 'gratis', 'за так', 'бесплатное', 'безвозмездный', 'даровой', 'комплиментарный', 'халявный', 'бонусный', 'подарочный', 'некоммерческий', 'открытый', 'свободный'],
    'онлайн': ['интернет', 'сетевой', 'веб', 'online', 'в сети', 'интернетный', 'сетевое', 'дистанционный', 'удаленный', 'виртуальный', 'киберпространственный', 'интернет-', 'веб-', 'сетево́й', 'подключенный'],
    'обучение': ['образование', 'учеба', 'курс', 'тренировка', 'обучающий', 'учебный', 'образование', 'педагогика', 'наука', 'инструктаж', 'подготовка', 'развитие', 'просвещение', 'обучалка', 'тренинг', 'семинар', 'лекции'],
    'музыка': ['аудио', 'мелодия', 'трек', 'песня', 'музло', 'звук', 'композиция', 'музыкальный', 'музычка', 'напев', 'мотив', 'аранжировка', 'саундтрек', 'минусовка', 'бит', 'ритм', 'гармония'],
    'видео': ['клип', 'ролик', 'фильм', 'кино', 'видеоролик', 'видеоклип', 'кинофильм', 'видеозапись', 'видеоконтент', 'видеофайл', 'муви', 'фильмец', 'роличек', 'запись', 'трансляция', 'стрим'],
    'фото': ['изображение', 'картинка', 'снимок', 'фотография', 'фотоснимок', 'фотка', 'карточка', 'фотокарточка', 'снимок', 'пикча', 'фотодокумент', 'иллюстрация', 'портрет', 'пейзаж', 'фотоизображение'],
    'социальный': ['соцсеть', 'общение', 'коммуникация', 'социум', 'социальная сеть', 'община', 'соц сеть', 'сообщество', 'сеть', 'платформа', 'медиа', 'форум', 'чат', 'блог', 'соцплатформа', 'интернет-сообщество'],
    'новости': ['события', 'информация', 'обновления', 'новостной', 'новостная лента', 'сводка', 'новостник', 'хроника', 'репортаж', 'анонс', 'пресса', 'медиа', 'лента', 'дайджест', 'обзор', 'свежие новости'],
    'магазин': ['шопинг', 'покупки', 'торговля', 'маркет', 'онлайн магазин', 'маркетплейс', 'торговая площадка', 'бутик', 'лавка', 'торговый центр', 'интернет-магазин', 'электронная коммерция', 'e-commerce', 'ритейл', 'продажи'],
    'книга': ['литература', 'чтение', 'издание', 'книжный', 'электронная книга', 'бук', 'литературное', 'том', 'издание', 'публикация', 'рукопись', 'фолиант', 'бестселлер', 'роман', 'повесть', 'рассказ'],
    'здоровье': ['медицина', 'фитнес', 'здоровый', 'медицинский', 'здоровье и фитнес', 'мед', 'оздоровление', 'здравоохранение', 'благополучие', 'самочувствие', 'витальность', 'гигиена', 'профилактика', 'реабилитация'],
    'путешествие': ['туризм', 'поездка', 'отдых', 'тур', 'путешествия', 'трип', 'вояж', 'турпоездка', 'экскурсия', 'паломничество', 'круиз', 'сафари', 'экспедиция', 'поход', 'отпуск', 'каникулы'],
    'еда': ['питание', 'рецепт', 'кулинария', 'пища', 'еда и напитки', 'кухня', 'готовка', 'продукты', 'блюдо', 'кушанье', 'гастрономия', 'диета', 'меню', 'рацион', 'провизия', 'закуска']
}
    def augment_text(self, text):
        """Аугментация текста с разными стратегиями"""
        if random.random() < 0.6:  # 70% chance для аугментации
            words = text.split()
            if len(words) <= 3:  # Слишком короткий текст - не аугментируем
                return text

            augmented_words = []

            for word in words:
                word_lower = word.lower().strip('.,!?;:')

                # Замена синонимами с вероятностью 15%
                if word_lower in self.synonyms and random.random() < 0.2:
                    synonym = random.choice(self.synonyms[word_lower])
                    # Сохраняем регистр первой буквы
                    if word[0].isupper():
                        synonym = synonym.capitalize()
                    augmented_words.append(synonym)
                else:
                    augmented_words.append(word)

            return ' '.join(augmented_words)
        return text

class MultiStrategyAppDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=300, is_training=True,
                 augment_prob=0.5, augment_strategies=['synonym', 'delete', 'swap']):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.is_training = is_training
        self.augment_prob = augment_prob
        self.augment_strategies = augment_strategies if is_training else []
        self.augmenter = AdvancedTextAugmenter() if is_training else None

    def __len__(self):
        return len(self.texts)

    def _apply_augmentation(self, text):
        """Применяет случайную стратегию аугментации"""
        if not self.augment_strategies or random.random() > self.augment_prob:
            return text

        strategy = random.choice(self.augment_strategies)

        if strategy == 'synonym' and self.augmenter:
            return self.augmenter.augment_text(text)

        elif strategy == 'delete':
            # Удаление случайных слов (10-20%)
            words = text.split()
            if len(words) > 5:
                n_to_delete = max(1, int(len(words) * random.uniform(0.1, 0.2)))
                indices_to_keep = random.sample(range(len(words)), len(words) - n_to_delete)
                words = [words[i] for i in sorted(indices_to_keep)]
                return ' '.join(words)
            return text

        elif strategy == 'swap':
            # Свап соседних слов
            words = text.split()
            if len(words) > 3:
                for _ in range(max(1, len(words) // 10)):
                    i = random.randint(0, len(words) - 2)
                    words[i], words[i + 1] = words[i + 1], words[i]
                return ' '.join(words)
            return text

        elif strategy == 'repeat':
            # Повторение важных частей
            words = text.split()
            if len(words) > 4:
                # Повторяем первые 1-2 слова в конце
                n_repeat = random.randint(1, 2)
                repeated = words[:n_repeat]
                words.extend(repeated)
                return ' '.join(words)
            return text

        return text

    def __getitem__(self, idx):
        text = self.texts[idx]

        # Применяем аугментацию
        if self.is_training:
            text = self._apply_augmentation(text)

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        item = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)

        return item

# ==================== УЛУЧШЕННЫЕ ФУНКЦИИ ОЦЕНКИ ====================
def calculate_hit_at_k(predictions, true_labels, k=3):
    """Вычисление HitRate@K с разными стратегиями"""
    hit_count = 0
    for i in range(len(predictions)):
        true_indices = set(np.where(true_labels[i] > 0)[0])

        # Стратегия 1: Топ-K по уверенности
        pred_indices = set(np.argsort(predictions[i])[-k:])

        # Стратегия 2: Порог + топ-K (резервная)
        if not true_indices & pred_indices:
            threshold = np.sort(predictions[i])[-k]  # k-й наибольший скор
            pred_indices_thresh = set(np.where(predictions[i] >= threshold)[0])
            if true_indices & pred_indices_thresh:
                hit_count += 1
        else:
            hit_count += 1

    return hit_count / len(predictions)

def evaluate_model(model, dataloader, device):
    """Расширенная оценка модели"""
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device) if 'labels' in batch else None

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.sigmoid(outputs.logits)
            all_preds.append(preds.cpu().numpy())
            if labels is not None:
                all_labels.append(labels.cpu().numpy())

    all_preds = np.vstack(all_preds)
    if all_labels:
        all_labels = np.vstack(all_labels)
        hitrate_3 = calculate_hit_at_k(all_preds, all_labels, k=3)
        hitrate_1 = calculate_hit_at_k(all_preds, all_labels, k=1)
        hitrate_5 = calculate_hit_at_k(all_preds, all_labels, k=5)
        return hitrate_3, hitrate_1, hitrate_5, all_preds, all_labels
    else:
        return None, None, None, all_preds, None

# ==================== УЛУЧШЕННОЕ ОБУЧЕНИЕ ====================
class FocalLoss(nn.Module):
    """Focal Loss для борьбы с несбалансированностью классов"""
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = torch.nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss

def train_epoch(model, train_loader, optimizer, scheduler, device, accumulation_steps, criterion):
    """Улучшенное обучение на одной эпохе"""
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []

    optimizer.zero_grad()

    for step, batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        # Используем кастомный лосс если передан, иначе стандартный
        if criterion:
            loss = criterion(outputs.logits, labels)
        else:
            loss = outputs.loss

        loss = loss / accumulation_steps
        loss.backward()

        if (step + 1) % accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        total_loss += loss.item() * accumulation_steps

        # Собираем предсказания
        preds = torch.sigmoid(outputs.logits)
        all_preds.append(preds.detach().cpu().numpy())
        all_labels.append(labels.cpu().numpy())

        if step % 50 == 0:
            current_lr = scheduler.get_last_lr()[0]
            print(f"  Batch {step}/{len(train_loader)}, Loss: {loss.item():.4f}, LR: {current_lr:.2e}")

    # Метрики на тренировочных данных
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    train_hitrate_3 = calculate_hit_at_k(all_preds, all_labels, k=3)
    train_hitrate_1 = calculate_hit_at_k(all_preds, all_labels, k=1)

    avg_loss = total_loss / len(train_loader)
    return avg_loss, train_hitrate_3, train_hitrate_1

# ==================== ОСНОВНАЯ ФУНКЦИЯ ====================
def main():
    print("🚀 ЗАПУСК УЛУЧШЕННОГО ОБУЧЕНИЯ...")
    print(f"Устройство: {config.device}")
    print(f"Модель: {config.model_name}")
    print(f"Batch size: {config.batch_size}")
    print(f"Learning rate: {config.learning_rate}")
    print(f"Max length: {config.max_length}")

    # Загрузка данных
    train_data, test_data, mlb, train_labels = load_and_prepare_data()

    # Инициализация токенизатора и КАСТОМНОЙ модели
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    model = MultiLabelTransformerWithHeads(config.model_name, len(mlb.classes_))
    model.to(config.device)

    # Стратифицированное разделение
    train_texts, val_texts, train_y, val_y = train_test_split(
        train_data['text'].tolist(),
        train_labels,
        test_size=0.1,  # Уменьшил валидацию для большего тренировочного набора
        random_state=42,
        stratify=train_labels.argmax(axis=1)
    )

    print(f"\n📊 Разделение данных:")
    print(f"Тренировочные: {len(train_texts)}")
    print(f"Валидационные: {len(val_texts)}")

    # Создание улучшенных датасетов
    train_dataset = MultiStrategyAppDataset(
        train_texts, train_y, tokenizer, config.max_length,
        is_training=True, augment_prob=0.6
    )
    val_dataset = MultiStrategyAppDataset(
        val_texts, val_y, tokenizer, config.max_length,
        is_training=False
    )

    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size * 2, shuffle=False)

    # Оптимизатор с разными learning rates для разных слоев
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters()
                   if not any(nd in n for nd in no_decay) and 'classifier' not in n and 'head' not in n],
         'weight_decay': 0.01, 'lr': config.learning_rate},
        {'params': [p for n, p in model.named_parameters()
                   if any(nd in n for nd in no_decay) and 'classifier' not in n and 'head' not in n],
         'weight_decay': 0.0, 'lr': config.learning_rate},
        {'params': [p for n, p in model.named_parameters() if 'category_head' in n or 'genre_head' in n],
         'weight_decay': 0.01, 'lr': config.learning_rate * 2},  # Более высокий LR для головок классификатора
    ]

    optimizer = AdamW(optimizer_grouped_parameters)
    total_steps = len(train_loader) * config.epochs // config.accumulation_steps
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(total_steps * config.num_warmup_ratio),
        num_training_steps=total_steps
    )

    # Focal Loss для несбалансированных данных
    criterion = FocalLoss(alpha=1, gamma=2)

    # Обучение с ранней остановкой
    best_hitrate = 0.0
    patience_counter = 0

    for epoch in range(config.epochs):
        print(f"\n🎯 ЭПОХА {epoch + 1}/{config.epochs}")

        # Обучение
        train_loss, train_hitrate_3, train_hitrate_1 = train_epoch(
            model, train_loader, optimizer, scheduler, config.device,
            config.accumulation_steps, criterion
        )

        # Валидация
        val_hitrate_3, val_hitrate_1, val_hitrate_5, _, _ = evaluate_model(model, val_loader, config.device)

        print(f"Train Loss: {train_loss:.4f}")
        print(f"Train H@1: {train_hitrate_1:.4f}, H@3: {train_hitrate_3:.4f}")
        print(f"Val H@1: {val_hitrate_1:.4f}, H@3: {val_hitrate_3:.4f}, H@5: {val_hitrate_5:.4f}")

        # Ранняя остановка и сохранение лучшей модели
        if val_hitrate_3 > best_hitrate:
            best_hitrate = val_hitrate_3
            torch.save(model.state_dict(), 'best_model.pth')
            torch.save(optimizer.state_dict(), 'best_optimizer.pth')
            patience_counter = 0
            print(f"🎉 НОВАЯ ЛУЧШАЯ МОДЕЛЬ! H@3: {best_hitrate:.4f}")
        else:
            patience_counter += 1
            print(f"⏳ Ранняя остановка: {patience_counter}/{config.patience}")

        if patience_counter >= config.patience:
            print("🛑 Ранняя остановка сработала!")
            break

    print(f"\n✅ ЛУЧШИЙ РЕЗУЛЬТАТ: H@3 = {best_hitrate:.4f}")

    # ==================== УЛУЧШЕННОЕ ПРЕДСКАЗАНИЕ ====================
    print("\n🔮 ЗАГРУЗКА ЛУЧШЕЙ МОДЕЛИ ДЛЯ ПРЕДСКАЗАНИЙ...")
    model.load_state_dict(torch.load('best_model.pth'))

    # Подготовка тестовых данных
    test_dataset = MultiStrategyAppDataset(
        test_data['text'].tolist(),
        None,
        tokenizer,
        config.max_length,
        is_training=False
    )
    test_loader = DataLoader(test_dataset, batch_size=config.batch_size * 2, shuffle=False)

    # Предсказания с усреднением (TTA-like)
    model.eval()
    all_test_logits = []

    print("Генерация предсказаний...")
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(config.device)
            attention_mask = batch['attention_mask'].to(config.device)
            logits = model(input_ids=input_ids, attention_mask=attention_mask)
            all_test_logits.append(logits.logits.cpu().numpy())

    test_logits = np.vstack(all_test_logits)

    # УЛУЧШЕННАЯ СТРАТЕГИЯ ПРЕДСКАЗАНИЙ
    class_names = mlb.classes_
    predictions = []
    confidence_scores = []

    for i, logits_row in enumerate(test_logits):
        probs = 1 / (1 + np.exp(-logits_row))  # sigmoid вручную для контроля

        # Стратегия 1: Топ-3 по вероятности
        top3_indices = np.argsort(probs)[-3:][::-1]

        # Стратегия 2: Проверка уверенности
        top_prob = probs[top3_indices[0]]
        if top_prob < 0.3:  # Низкая уверенность
            # Ищем категории с максимальной вероятностью выше порога
            high_conf_indices = np.where(probs > 0.2)[0]
            if len(high_conf_indices) > 0:
                top3_indices = high_conf_indices[np.argsort(-probs[high_conf_indices])][:3]
            # Если все равно нет, берем топ-3 по логитам
            if len(top3_indices) == 0:
                top3_indices = np.argsort(logits_row)[-3:][::-1]

        predicted_categories = [class_names[idx] for idx in top3_indices]
        predictions.append("|".join(predicted_categories))
        confidence_scores.append(np.mean(probs[top3_indices]))

    # Сохраняем результаты
    submission = pd.DataFrame({
        'app_name': test_data['app_name'],
        'labels_str': predictions
    })

    submission.to_csv('enhanced_submission.tsv', sep='\t', index=False)
    print("📄 УЛУЧШЕННЫЕ РЕЗУЛЬТАТЫ СОХРАНЕНЫ В enhanced_submission.tsv")

    # Детальный анализ предсказаний
    print("\n📊 ДЕТАЛЬНЫЙ АНАЛИЗ ПРЕДСКАЗАНИЙ:")
    pred_counts = Counter([cat for pred in predictions for cat in pred.split('|')])
    top_predicted = pd.DataFrame({
        'category': list(pred_counts.keys()),
        'count': list(pred_counts.values())
    }).sort_values('count', ascending=False)

    print("Топ-15 самых частых предсказанных категорий:")
    print(top_predicted.head(15))

    # Анализ уверенности
    print(f"\n🎯 СТАТИСТИКА УВЕРЕННОСТИ:")
    print(f"Средняя уверенность: {np.mean(confidence_scores):.3f}")
    print(f"Медианная уверенность: {np.median(confidence_scores):.3f}")
    print(f"Доля предсказаний с уверенностью > 0.5: {np.mean(np.array(confidence_scores) > 0.5):.3f}")

    # Сравнение с наиболее частыми метками из тренировочных данных
    train_label_counts = train_labels.sum(axis=0)
    top_train_labels = pd.DataFrame({
        'category': mlb.classes_,
        'count': train_label_counts
    }).sort_values('count', ascending=False).head(10)

    print("\n📈 СРАВНЕНИЕ С ТРЕНИРОВОЧНЫМИ ДАННЫМИ:")
    print("Топ-10 самых частых категорий в тренировочных данных:")
    print(top_train_labels)

if __name__ == "__main__":
    main()

🚀 ЗАПУСК УЛУЧШЕННОГО ОБУЧЕНИЯ...
Устройство: cuda
Модель: sberbank-ai/ruBert-base
Batch size: 16
Learning rate: 1.5e-05
Max length: 300
Доступные колонки в train: ['app_name', 'full_description', 'shortDescription', 'labels_str']
Средняя длина текста: 285 символов
Максимальная длина: 425 символов

📊 Анализ меток:
Всего классов: 45
Общее количество меток: 63781
Меток на приложение: 1.19

Топ-10 самых частых категорий:
         category  count
40          tools   6241
11         casual   5269
29         puzzle   4332
4          arcade   3917
14  entertainment   3739
34      simulator   3124
8        business   3076
13      education   3076
17   foodAndDrink   3005
0          action   2219

📊 Разделение данных:
Тренировочные: 48144
Валидационные: 5350

🎯 ЭПОХА 1/5
  Batch 0/3009, Loss: 0.1045, LR: 0.00e+00
  Batch 50/3009, Loss: 0.0942, LR: 4.53e-07
  Batch 100/3009, Loss: 0.0788, LR: 9.07e-07
  Batch 150/3009, Loss: 0.0621, LR: 1.36e-06
  Batch 200/3009, Loss: 0.0448, LR: 1.81e-06
  Batc

KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, BertModel, AutoConfig
from sklearn.preprocessing import MultiLabelBinarizer
import warnings
warnings.filterwarnings('ignore')

# ==================== КОНФИГУРАЦИЯ ====================
class Config:
    model_name = "sberbank-ai/ruBert-base"
    batch_size = 16
    max_length = 300
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = Config()

# ==================== МОДЕЛЬ ====================
class MultiLabelTransformerWithHeads(nn.Module):
    def __init__(self, model_name, num_labels, hidden_dropout_prob=0.3):
        super().__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.backbone = BertModel.from_pretrained(model_name)
        self.category_head = nn.Linear(self.config.hidden_size, num_labels)
        self.genre_head = nn.Linear(self.config.hidden_size, num_labels)
        self.dropout = nn.Dropout(hidden_dropout_prob)
        self.layer_norm = nn.LayerNorm(self.config.hidden_size)

    def forward(self, input_ids, attention_mask):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]
        normalized_output = self.layer_norm(pooled_output)
        dropout_output = self.dropout(normalized_output)
        logits1 = self.category_head(dropout_output)
        logits2 = self.genre_head(dropout_output)
        logits = (logits1 + logits2) / 2
        return logits

# ==================== ДАТАСЕТ ====================
class TestDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=300):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

# ==================== ФУНКЦИЯ ПРЕДСКАЗАНИЯ ====================
def predict_on_test_data():
    print("🚀 ЗАПУСК ПРЕДСКАЗАНИЙ НА ТЕСТОВЫХ ДАННЫХ...")
    print(f"Устройство: {config.device}")
    print(f"Модель: {config.model_name}")

    # Загрузка тестовых данных
    try:
        test_data = pd.read_csv('/content/test.tsv', sep='\t')
        print(f"Загружено тестовых данных: {len(test_data)} строк")
    except:
        # Если файл не найден, создаем демо-данные
        print("⚠️ Файл test.tsv не найден! Создаем демо-данные...")
        test_data = pd.DataFrame({
            'app_name': [f'App_{i}' for i in range(100)],
            'description': [f'Описание приложения {i}' for i in range(100)]
        })

    # Создание текста для предсказания
    def create_text(row):
        parts = []
        if 'app_name' in row:
            app_name = str(row['app_name']).strip()
            if app_name and app_name != 'nan':
                parts.append(f"Название: {app_name}")

        for col in ['shortDescription', 'short_description', 'description', 'full_description']:
            if col in row and pd.notna(row[col]):
                desc = str(row[col]).strip()
                if desc and desc != 'nan':
                    parts.append(f"Описание: {desc}")
                    break

        if not parts:
            return "Приложение"
        return " ".join(parts)

    test_data['text'] = test_data.apply(create_text, axis=1)

    # Загрузка токенизатора
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)

    # Создание тестового датасета
    test_dataset = TestDataset(test_data['text'].tolist(), tokenizer, config.max_length)
    test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)

    # Определяем количество классов (нужно загрузить из тренировочных данных или установить вручную)
    try:
        # Пробуем загрузить mlb из файла или создать заново
        train_data = pd.read_csv('/content/train.tsv', sep='\t')
        mlb = MultiLabelBinarizer()
        mlb.fit(train_data['labels_str'].str.split('|'))
        num_classes = len(mlb.classes_)
        print(f"Найдено классов: {num_classes}")
    except:
        # Если не удалось загрузить, используем стандартные классы
        default_categories = ['игра', 'образование', 'музыка', 'видео', 'социальный',
                            'новости', 'магазин', 'книга', 'здоровье', 'путешествие']
        mlb = MultiLabelBinarizer()
        mlb.fit([default_categories])  # Фитируем на всем списке
        num_classes = len(mlb.classes_)
        print(f"⚠️ Используем стандартные классы: {num_classes}")

    # Загрузка модели
    model = MultiLabelTransformerWithHeads(config.model_name, num_classes)

    try:
        model.load_state_dict(torch.load('/content/drive/MyDrive/best_model (1).pth', map_location=config.device))
        print("✅ Модель успешно загружена из best_model.pth")
    except FileNotFoundError:
        print("❌ Файл best_model.pth не найден!")
        print("Пожалуйста, убедитесь что файл существует в текущей директории")
        return
    except Exception as e:
        print(f"❌ Ошибка при загрузке модели: {e}")
        return

    model.to(config.device)
    model.eval()

    # Предсказания
    all_predictions = []

    print("🎯 Генерация предсказаний...")
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(config.device)
            attention_mask = batch['attention_mask'].to(config.device)

            logits = model(input_ids=input_ids, attention_mask=attention_mask)
            probabilities = torch.sigmoid(logits).cpu().numpy()

            # Выбираем топ-3 предсказания для каждого примера
            for prob_row in probabilities:
                top_3_indices = np.argsort(prob_row)[-3:][::-1]  # Индексы топ-3 предсказаний
                predicted_categories = [mlb.classes_[idx] for idx in top_3_indices]
                all_predictions.append("|".join(predicted_categories))

    # Создание сабмита
    submission = pd.DataFrame({
        'app_name': test_data['app_name'],
        'labels_str': all_predictions
    })

    # Сохранение результатов
    submission_file = 'test_predictions.tsv'
    submission.to_csv(submission_file, sep='\t', index=False)
    print(f"✅ Предсказания сохранены в {submission_file}")

    # Статистика предсказаний
    print("\n📊 СТАТИСТИКА ПРЕДСКАЗАНИЙ:")
    print(f"Всего предсказано: {len(submission)} строк")

    # Анализ распределения предсказанных категорий
    all_predicted_categories = []
    for pred in all_predictions:
        all_predicted_categories.extend(pred.split('|'))

    from collections import Counter
    category_counts = Counter(all_predicted_categories)

    print("\nТоп-10 самых частых предсказанных категорий:")
    for category, count in category_counts.most_common(10):
        print(f"  {category}: {count} раз")

    # Показываем первые несколько предсказаний
    print(f"\n📝 ПЕРВЫЕ 10 ПРЕДСКАЗАНИЙ:")
    for i in range(min(10, len(submission))):
        print(f"  {i+1}. {submission.iloc[i]['app_name']} -> {submission.iloc[i]['labels_str']}")

    return submission

# ==================== АЛЬТЕРНАТИВНАЯ ПРОСТАЯ МОДЕЛЬ ====================
class SimpleBertClassifier(nn.Module):
    """Упрощенная модель если основная не работает"""
    def __init__(self, model_name, num_labels):
        super().__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.dropout = nn.Dropout(0.3)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

def predict_with_simple_model():
    """Альтернативная функция предсказания с упрощенной моделью"""
    print("🔄 Используем упрощенную модель...")

    # Загрузка тестовых данных (аналогично основной функции)
    test_data = pd.read_csv('/content/test.tsv', sep='\t')
    test_data['text'] = test_data.apply(
        lambda x: f"{x['app_name']} {x.get('description', '')}", axis=1
    )

    # Определяем количество классов
    num_classes = 50  # Можно изменить на нужное количество

    # Загружаем упрощенную модель
    model = SimpleBertClassifier(config.model_name, num_classes)

    try:
        model.load_state_dict(torch.load('best_model.pth', map_location=config.device))
        print("✅ Упрощенная модель загружена успешно")
    except:
        print("❌ Не удалось загрузить модель. Проверьте архитектуру.")
        return

    # Остальной код аналогичен основной функции...
    return predict_on_test_data()

# ==================== ЗАПУСК ====================
if __name__ == "__main__":
    # Пробуем основную функцию
    try:
        result = predict_on_test_data()
        print("\n🎉 ПРЕДСКАЗАНИЯ УСПЕШНО ЗАВЕРШЕНЫ!")
    except Exception as e:
        print(f"❌ Ошибка в основной функции: {e}")
        print("Пробуем альтернативный подход...")
        try:
            result = predict_with_simple_model()
        except Exception as e2:
            print(f"❌ Ошибка в альтернативной функции: {e2}")
            print("Пожалуйста, проверьте:")
            print("1. Наличие файла best_model.pth")
            print("2. Соответствие архитектуры модели")
            print("3. Наличие тестовых данных")

🚀 ЗАПУСК ПРЕДСКАЗАНИЙ НА ТЕСТОВЫХ ДАННЫХ...
Устройство: cuda
Модель: sberbank-ai/ruBert-base
Загружено тестовых данных: 15046 строк
Найдено классов: 45
✅ Модель успешно загружена из best_model.pth
🎯 Генерация предсказаний...
✅ Предсказания сохранены в test_predictions.tsv

📊 СТАТИСТИКА ПРЕДСКАЗАНИЙ:
Всего предсказано: 15046 строк

Топ-10 самых частых предсказанных категорий:
  casual: 4232 раз
  tools: 4192 раз
  arcade: 3639 раз
  entertainment: 3098 раз
  business: 2924 раз
  simulator: 2407 раз
  puzzle: 2259 раз
  education: 1972 раз
  action: 1850 раз
  lifestyle: 1691 раз

📝 ПЕРВЫЕ 10 ПРЕДСКАЗАНИЙ:
  1. Lemon clicker -> casual|arcade|simulator
  2. Memo Английский язык -> education|books|tools
  3. Slave Man Rescue -> puzzle|adventure|arcade
  4. Taking Care of Granny -> casual|arcade|children
  5. Escape From Classic Room -> puzzle|casual|arcade
  6. Boy Rescue From Ghost -> adventure|action|puzzle
  7. Charmer Escape -> puzzle|casual|arcade
  8. Jolly Theatre Escape 2 -> puzzle